# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testting part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file ’housing.names’ for more information on the attributes.

In [275]:
import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from sklearn.datasets import load_boston
boston=load_boston()
testfile = urllib.request.URLopener()
testfile.retrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names", "housing.names")
df=pd.DataFrame(boston.data)
df.columns=['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X=boston.data
y=boston.target


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [276]:
df.head(10)

,crime_rate,res_land_zoned,industry,charles_river,nox,avg_num_rooms,prop_bf_1940,dst_emply_center,rd_highway_idx,tax_rate,stdnt_tchr_ratio,prop_blacks,low_status_pct
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


### Exercise 1

Write the *split_train_test(X,y,split,seed)*, given an instance matrix $X \in \mathbb{R}^{N \times D}$, labels $y \in Y^N$, a split ratio in $[0, 1]$ and a random seed $\in \mathbb{Z}$. Split the dataset in $(split×100)\%$ of the instances for training our model and the rest for testing, i.e. 

$$ \left|X_{\text{train}}\right| = \lceil \text{split} \cdot N \rceil, \qquad |X_{\text{train}}| + |X_{\text{test}}| = N. $$
Make sure you use the given random number generator seed so we all get the same results. The function is supposed to return:

- X_train, y_train: the training instances and labels;
- X_test, y_test: the test instances and labels,

in the same order as was mentioned.

Hint: It may be helpful to use shuffling functionality (e.g. np.random.shuffle).

In [277]:
def split_train_test(X,y,split,seed):
    Xy = []
    length = len(y)
    for i in range(length):
        xyi = np.append(X[i], y[i])
        Xy.append(xyi)

    random = np.random
    random.seed(seed)
    random.shuffle(Xy)

    split_point = round(length * split)

    X_train, y_train, X_test, y_test = [],[],[],[]

    for i in range(length):
        if i <= split_point:
            X_train.append(Xy[i][0:length-1])
            y_train.append(Xy[i][-1])
        elif i > split_point:
            X_test.append(Xy[i][0:length-1])
            y_test.append(Xy[i][-1])

    return X_train, y_train, X_test, y_test

### Exercise 2

Write the function *train_linear_reg(X_train,y_train,lmbd)*.
Implement the ridge regression model (slide 24). The function should output the learned weight vector $\theta \in \mathbb{R}^D$ or $\mathbb{R}^{D+1}$ (depending on whether you are adding *bias*).

In [278]:
def train_linear_reg(X, y, lmbd):
    Xm = np.array(X)
    num_rows, num_cols = Xm.shape
    idtt = np.identity(num_cols)
    return np.matmul(np.linalg.inv(np.matmul(Xm.T, Xm) + (lmbd * idtt)), np.matmul(Xm.T, np.array(y)))

### Exercise 3

Write the function *predict(X,theta)* which predicts housing values vector pred for a dataset X and a previously trained parameter vector $\theta$.

In [279]:
def predict(X, theta):
    predictions = []
    for x in X:
        x_weighted_sum = 0
        for i in range(x.size):
            x_weighted_sum += theta[i] * x[i]
        predictions.append(x_weighted_sum)
    return predictions


### Exercise 4

Write the function *mean_abs_loss(y_true,y_pred)* which computes the mean of the absolute differences between our prediction vector $y\_pred$ and the real housing values $y\_true$.

In [280]:
def mean_abs_loss(y_true,y_pred):
    return abs(np.subtract(y_true, y_pred)).sum() / len(y_true)

[18.19973576339029, 19.300171084967403, 26.700775189762524, 19.200316947311865, 30.099505257135842, 20.60031466027916, 49.995667932590095, 18.69983672221291, 20.60054550119304, 31.100054664022505, 14.000366877209265, 17.79922254678399, 42.299376930292944, 15.300463582635313, 18.500184796367314, 21.399876323712085, 15.000620992275618, 20.70056518013236, 21.40026012717339, 21.70004744627544, 22.000717094448653, 31.59998196446175, 22.000738556277998, 10.199494260917454, 22.599678148146182, 20.00015437878261, 17.801059617536094, 13.599994412925698, 11.800192556241266, 19.400401803778347, 21.40019917980904, 32.89983943969797, 20.800164859191053, 31.00046318036674, 17.500058732206686, 15.400128665297474, 10.800203645621648, 34.699521401891054, 25.000300498127096, 48.79920402072113, 42.79904234049661, 19.500157461873165, 30.100354283601348, 22.20019338591655, 49.997433557015036, 23.100233959749115, 32.499514138568024, 19.59972649053487, 14.900390199321793, 26.400175036124697, 36.9991369576550

### Exercise 5

Evaluate your solutions by running the following code. 

Moreover, answer the following questions: What is the most important feature in your model? Are there features that are not so important? What happens if you remove them? Are there outliers with a high absolute loss?

In [281]:
seed = 3
lmbd=1
split=0.7
X_train,y_train,X_test,y_test=split_train_test(X,y,split,seed)
theta=train_linear_reg(X_train,y_train,lmbd)
y_pred=predict(X_test,theta)
mae=mean_abs_loss(y_test,y_pred)
print ('The mean absolute loss is {loss:0.3f}'.format(loss=mae*1000))

The mean absolute loss is 0.408
